In [1]:
import re
from datetime import date
import pickle
import seaborn as sns
import pandas as pd

#import toolboxes
from reddit_toolbox import piecewise_lin_fit, search_mentions, search_drug_pattern, function_find_pattern, moving_average

# Getting data:
with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
     reddit_df = pickle.load(f)
        
drug_kwrds = {'4-FA' : '4-FA|4FMP', 
              '4-MMC': '4-MMC|mephedrone | mefedron',
              '6-ABP': '6-APB| Benzofury| Benzo Fury',
              'A-PHP'  : 'alpha-PHP|α-PHP|a-php',
              'Phenibut': 'Phenibut|fenibut',

             }
reddit_df['year'] = pd.to_datetime(reddit_df['created_utc']).dt.strftime('%Y')
reddit_df['year'] = reddit_df['year'].astype('int')


,title,selftext,body,contains
0,https://3mmc.sfsdf.sfdsf.sdfsf,NaN,NaN,False
1,3mmc,NaN,NaN,True
2,3 mmc,NaN,NaN,True
3,3MmC,NaN,NaN,True
4,,3mmc,NaN,True
5,,NaN,3mmc,True
6,,NaN,4 abc,True


In [7]:
def search_drug(drug_df=reddit_df, drug='', beg_year=2012, end_year=2022):
    
    search_mentions(drug_df, drug_name=function_find_pattern(drug, drug_kwrds) )
    df_no_comments = drug_df[drug_df['kind']=='t3'].reset_index()
    df_no_comments = df_no_comments[(df_no_comments['year']>=beg_year) & (df_no_comments['year']<=end_year)]

    df_no_comments = df_no_comments.drop('year', axis=1)
    agg_data = df_no_comments.groupby('year_month').sum()
    
    agg_data['moving-avg-mentions'] = moving_average(agg_data, 'contains', window=5)


    return agg_data.reset_index()    

In [8]:
search_drug(drug_df=reddit_df, drug='3 mmc', beg_year=2012, end_year=2022)

,year_month,contains,moving-avg-mentions
0,2012-01-01,0,0.0
1,2012-07-01,0,0.0
2,2012-09-01,0,0.0
3,2013-04-01,0,0.0
4,2013-05-01,0,0.0
...,...,...,...
113,2022-08-01,1,0.6
114,2022-09-01,6,1.4
115,2022-10-01,2,1.8
116,2022-11-01,26,7.0


In [ ]:
import matplotlib.pyplot as plt
sns.set()
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16, 8)
sns.set(style="ticks")
sns.lineplot(x=data.index,
            y = data['moving-avg-mentions'],
            estimator=sum,
            ci=None,
            color='b',
            )
